In [8]:
import numpy as np
from scipy.interpolate import interp1d
import time
import redshifts
from math import lgamma
import loginterp
import common as c
import copy
from scipy.linalg import cholesky
from scipy import linalg
import healpy as hp

In [1]:
import config as conf
import estim
from estim import estimator
data_lmax = 4000

nside = 20
lmax = 4000

estimator = estimator(data_lmax, conf)


Cls = estimator.load_theory_Cl('vr', 'vr')
print(Cls.shape)
print('Generating vmap and vlm')
vmap, vlm = estimator.get_maps_and_alms(['vr'], nside, lmax)

Default lss = 'g' . Modify with set_lss method.
Default N_fine_modes = 8. Modify with set_Nfine method.
(95, 8, 8)
Generating vmap and vlm
cholesky
formatting covmat
generating realization


In [15]:
vmap[0].shape

(8, 4800)

In [16]:
vmap_test = hp.alm2map(vlm[0][0], nside)

In [ ]:
vmap